In [1]:
import pybamm
import pandas as pd
import os
import numpy as np

os.chdir(pybamm.__path__[0])

pybamm.set_logging_level("INFO")

# load model and update parameters so the input current is the US06 drive cycle
model = pybamm.lithium_ion.SPMe({"thermal": "lumped"})
param = model.default_parameter_values

# import drive cycle from file
drive_cycle = pd.read_csv(
"input/drive_cycles/US06.csv", comment="#", header=None
).to_numpy()

drive_step = pybamm.step.current(value=drive_cycle)
experiment = pybamm.Experiment(operating_conditions=[drive_step])

# create and run simulation using the CasadiSolver in "fast" mode, remembering to
# pass in the updated parameters
sim = pybamm.Simulation(
model, parameter_values=param, experiment=experiment, solver=pybamm.CasadiSolver(mode="fast")
)

# we need to specify the solve time to be different from the default [0, 600]
# this will raise a warning but this can be ignored
sim.solve()

# use this instead to get rid of the warning
t_drive = drive_cycle[:, 0]
step_size = np.min(np.diff(t_drive))
t_eval = np.arange(0, 1800, step_size)
sim.solve(t_eval)
# you can play with step_size to change the level of resolution of the drive cycle
sim.plot(
[
"Negative particle surface concentration [mol.m-3]",
"Electrolyte concentration [mol.m-3]",
"Positive particle surface concentration [mol.m-3]",
"Current [A]",
"Negative electrode potential [V]",
"Electrolyte potential [V]",
"Positive electrode potential [V]",
"Terminal voltage [V]",
"X-averaged cell temperature [K]",
]
)

2024-07-07 15:24:04.765 - [INFO] base_model._build_model(652): Start building Single Particle Model with electrolyte
2024-07-07 15:24:04.915 - [INFO] base_battery_model.build_model(1073): Finish building Single Particle Model with electrolyte
2024-07-07 15:24:04.940 - [INFO] callbacks.on_experiment_start(164): Start running experiment
2024-07-07 15:24:04.942 - [INFO] parameter_values.process_model(394): Start setting parameters for Single Particle Model with electrolyte
2024-07-07 15:24:05.156 - [INFO] parameter_values.process_model(463): Finish setting parameters for Single Particle Model with electrolyte
2024-07-07 15:24:05.160 - [INFO] discretisation.process_model(145): Start discretising Single Particle Model with electrolyte
2024-07-07 15:24:05.185 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing variable Discharge capacity [A.h] from rhs
2024-07-07 15:24:05.188 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing variable Thro

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…